# CodeLlama fine-tuning for JIT-SDP

This script is inspired by https://www.analyticsvidhya.com/blog/2024/06/finetuning-llama-3-for-sequence-classification/

In [ ]:
!pip install -q transformers accelerate trl bitsandbytes datasets evaluate huggingface-cli
!pip install -q peft scikit-learn

ERROR: Could not find a version that satisfies the requirement huggingface-cli (from versions: none)
ERROR: No matching distribution found for huggingface-cli
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import pickle
import torch

In [ ]:
filename = "data/changes_test_lst.pkl"

In [ ]:
import pickle

with open(filename, 'rb') as f:
    changes_test = pickle.load(f)

In [ ]:
filename = "data/features_test.pkl"

In [ ]:
with open(filename, 'rb') as f:
    features_test = pickle.load(f)

<ipython-input-6-972c5609ada2>:2: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  features_test = pickle.load(f)


In [ ]:
def get_exp_desc(mean, value):
    if int(float(value)) == int(mean):
        return 'average'
    elif float(value) < mean:
        return 'below average'
    elif float(value) > mean:
        return 'above average'
    else:
        return 'average'

In [ ]:
filename = "data/features_train.pkl"

In [ ]:
import pickle

with open(filename, 'rb') as f:
    features_train = pickle.load(f)

<ipython-input-9-7104eed7509f>:4: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  features_train = pickle.load(f)


In [ ]:
exp_mean, rexp_mean, sexp_mean = features_train['exp'].astype(float).mean(), features_train['rexp'].astype(float).mean(), features_train['sexp'].astype(float).mean()

In [ ]:
test_texts = []

for i in range(len(changes_test[0])):
    added_lines = ""
    if len(changes_test[3][i]["added_code"]) > 0:
        added_lines = "ADDED LINES: " + str(changes_test[3][i]["added_code"])

    removed_lines = ""
    if len(changes_test[3][i]["removed_code"]) > 0:
        removed_lines = "REMOVED LINES: " + str(changes_test[3][i]["removed_code"])

    exp_prompt = f"""\n
    MODIFIED SUBSYSTEMS: {features_test.loc[i,'ns']}
    MODIFIED DIRECTORIES: {features_test.loc[i,'nd']}
    MODIFIED FILES: {features_test.loc[i,'nf']}
    ENTROPY: {features_test.loc[i,'entropy']}
    LINES OF CODE ADDED: {features_test.loc[i,'la']}
    LINES OF CODE DELETED: {features_test.loc[i,'ld']}
    LINES OF CODE IN A FILE BEFORE THE CHANGE: {features_test.loc[i,'lt']}
    FIX: {features_test.loc[i,'fix']}
    DEVELOPERS THAT CHANGED THE MOFIFIED FILES: {features_test.loc[i,'ndev']}
    AVERAGE TIME BETWEEN THE LAST AND THE CURRENT CHANGE: {features_test.loc[i,'age']}
    UNIQUE CHANGES TO THE MODIFIED FILES: {features_test.loc[i,'nuc']}
    DEVELOPER EXPERIENCE: {get_exp_desc(exp_mean, features_test.loc[i,'exp'])}
    RECENT DEVELOPER EXPERIENCE: {get_exp_desc(rexp_mean, features_test.loc[i,'rexp'])}
    DEVELOPER EXPERIANCE ON THE CURRENT SUBSYSTEM: {get_exp_desc(sexp_mean, features_test.loc[i,'sexp'])}
    """

    text = f"""COMMIT MESSAGE: {changes_test[2][i]}
              {added_lines}
               {removed_lines}
               {exp_prompt}
               """

    test_texts.append(text)

In [ ]:
y_true_test = []

for i in range(len(changes_test[0])):
    y_true_test.append(changes_test[1][i])

In [ ]:
import pandas as pd

test_df = pd.DataFrame({'text': test_texts, 'label': y_true_test})

In [ ]:
from tqdm import tqdm

def generate_predictions(model,df_test):
    sentences = df_test.text.tolist()
    batch_size = 32
    all_outputs = []

    for i in tqdm(range(0, len(sentences), batch_size)):

        batch_sentences = sentences[i:i + batch_size]

        inputs = tokenizer(batch_sentences, return_tensors="pt",
        padding=True, truncation=True, max_length=512)

        inputs = {k: v.to('cuda' if torch.cuda.is_available() else 'cpu')
        for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)
            all_outputs.append(outputs['logits'])

    final_outputs = torch.cat(all_outputs, dim=0)
    probs = []
    for i in range(len(final_outputs)):
        probs.append(torch.softmax(final_outputs[i], dim=0)[1].cpu().numpy())
    print(probs)

    df_test['pred_prob']=probs
    df_test['predictions']=final_outputs.argmax(axis=1).cpu().numpy()

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import balanced_accuracy_score, classification_report, f1_score, precision_score, recall_score, roc_auc_score, auc, roc_curve

def get_metrics_result(test_df):
    y_test = test_df.label
    y_pred = test_df.predictions

    print("Classification Report:")
    print(classification_report(y_test, y_pred))

    print("Balanced Accuracy Score:", balanced_accuracy_score(y_test, y_pred))
    print("Accuracy Score:", accuracy_score(y_test, y_pred))

    f1 = f1_score(y_test, y_pred, average="binary")
    precision = precision_score(y_test, y_pred, average="binary")
    recall = recall_score(y_test, y_pred, average="binary")

    pred_prob = test_df.pred_prob
    fpr, tpr, thres = roc_curve(y_test, pred_prob)
    auc_score = auc(fpr, tpr)

    #roc_auc = roc_auc_score(y_test, y_pred)

    # Calculate G-Mean
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    g_mean = np.sqrt((tp / (tp + fn)) * (tn / (tn + fp)))
    r1 = tp / (tp + fn)
    assert r1 == recall
    r0 = tn / (tn + fp)

    print("F1 Score:", f1)
    print("Precision:", precision)
    print("Recall:", recall)
    #print("ROC AUC Score:", roc_auc)
    print("ROC AUC Score:", auc_score)

    print("R0:", r0)
    print("R1:", r1)
    print("G-Mean:", g_mean)

    return auc_score, g_mean


In [ ]:
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/
  Using cached bitsandbytes-0.45.5-py3-none-manylinux_2_24_x86_64.whl.metadata (5.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 29.0 MB/s eta 0:00:00


In [ ]:
token = <HF TOKEN>

In [ ]:
import torch
from transformers import BitsAndBytesConfig, AutoModelForSequenceClassification

quantization_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_use_double_quant = True,
    bnb_4bit_compute_dtype = torch.bfloat16
)

model_name = "meta-llama/CodeLlama-7b-hf"

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    num_labels=2,
    device_map='auto', token=token
)

config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/CodeLlama-7b-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True,
                                          token=token)

tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [ ]:
model.to('cuda')
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False
model.config.pretraining_tp = 1

## Model loading (after fine-tuning)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls -l /content/drive/MyDrive/UFPE/Tese/code_llama/auto/trained_weigths

total 69649
-rw------- 1 root root      779 Apr 27 02:51 adapter_config.json
-rw------- 1 root root 67176168 Apr 27 02:51 adapter_model.safetensors
drwx------ 2 root root     4096 Apr 27 02:51 checkpoint-1024
-rw------- 1 root root     5100 Apr 27 02:51 README.md
drwx------ 2 root root     4096 Apr 27 00:34 runs
-rw------- 1 root root      538 Apr 27 02:51 special_tokens_map.json
-rw------- 1 root root     1897 Apr 27 02:51 tokenizer_config.json
-rw------- 1 root root  3620830 Apr 27 02:51 tokenizer.json
-rw------- 1 root root   500058 Apr 27 02:51 tokenizer.model
-rw------- 1 root root     5432 Apr 27 02:51 training_args.bin


In [ ]:
from peft import PeftModel

model_name = "/content/drive/MyDrive/UFPE/Tese/code_llama/auto/trained_weigths"

model = PeftModel.from_pretrained(model, model_name)

In [ ]:
!pip install -U bitsandbytes

In [ ]:
import numpy as np

generate_predictions(model,test_df)
get_metrics_result(test_df)

100%|██████████| 172/172 [04:43<00:00,  1.65s/it]


[array(0.664, dtype=float16), array(0.802, dtype=float16), array(0.81, dtype=float16), array(0.8564, dtype=float16), array(0.926, dtype=float16), array(0.57, dtype=float16), array(0.1299, dtype=float16), array(0.7915, dtype=float16), array(0.8286, dtype=float16), array(0.1793, dtype=float16), array(0.929, dtype=float16), array(0.321, dtype=float16), array(0.1937, dtype=float16), array(0.1538, dtype=float16), array(0.845, dtype=float16), array(0.9565, dtype=float16), array(0.944, dtype=float16), array(0.6353, dtype=float16), array(0.908, dtype=float16), array(0.0981, dtype=float16), array(0.9507, dtype=float16), array(0.9844, dtype=float16), array(0.11945, dtype=float16), array(0.0453, dtype=float16), array(0.995, dtype=float16), array(0.995, dtype=float16), array(0.857, dtype=float16), array(0.2129, dtype=float16), array(0.6084, dtype=float16), array(0.991, dtype=float16), array(0.9297, dtype=float16), array(0.4219, dtype=float16), array(0.04694, dtype=float16), array(0.8765, dtype=flo

(np.float64(0.8561859193438142), np.float64(0.6965677159406639))

## Training data preparation

In [ ]:
filename = "data/changes_train_lst.pkl"

In [ ]:
import pickle

with open(filename, 'rb') as f:
    changes_train = pickle.load(f)

In [ ]:
filename = "data/changes_valid_lst.pkl"

In [ ]:
with open(filename, 'rb') as f:
    changes_eval = pickle.load(f)

In [ ]:
generate_predictions(model,test_df)


100%|██████████| 172/172 [04:01<00:00,  1.40s/it]


[array(0.4597, dtype=float16), array(0.416, dtype=float16), array(0.893, dtype=float16), array(0.8564, dtype=float16), array(0.719, dtype=float16), array(0.745, dtype=float16), array(0.659, dtype=float16), array(0.506, dtype=float16), array(0.8213, dtype=float16), array(0.1453, dtype=float16), array(0.0867, dtype=float16), array(0.7036, dtype=float16), array(0.632, dtype=float16), array(0.521, dtype=float16), array(0.5864, dtype=float16), array(0.705, dtype=float16), array(0.668, dtype=float16), array(0.1458, dtype=float16), array(0.3474, dtype=float16), array(0.7017, dtype=float16), array(0.6016, dtype=float16), array(0.662, dtype=float16), array(0.6226, dtype=float16), array(0.723, dtype=float16), array(0.05807, dtype=float16), array(0.6777, dtype=float16), array(0.1071, dtype=float16), array(0.5713, dtype=float16), array(0.882, dtype=float16), array(0.701, dtype=float16), array(0.756, dtype=float16), array(0.8276, dtype=float16), array(0.676, dtype=float16), array(0.3262, dtype=floa

In [ ]:
test_df.head()

,text,label,pred_prob,predictions
0,COMMIT MESSAGE: FIX : Invalid URL when using d...,1.0,0.4597,0
1,COMMIT MESSAGE: IMPROVEMENT : Parse license in...,1.0,0.416,0
2,COMMIT MESSAGE: IMPROVEMENT : write home page ...,1.0,0.893,1
3,COMMIT MESSAGE: FIX : Ivy ibiblio resolver cho...,1.0,0.8564,1
4,COMMIT MESSAGE: fix style\n ADDED...,1.0,0.719,1


In [ ]:
import numpy as np

In [ ]:
get_metrics_result(test_df)

### Semantic features + expert features

In [ ]:
filename = "data/features_valid.pkl"

In [ ]:
with open(filename, 'rb') as f:
    features_eval = pickle.load(f)

<ipython-input-22-b6d822b26641>:2: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  features_eval = pickle.load(f)


In [ ]:
train_texts = []

for i in range(len(changes_train[0])):
    added_lines = ""
    if len(changes_train[3][i]["added_code"]) > 0:
        added_lines = "ADDED LINES: " + str(changes_train[3][i]["added_code"])

    removed_lines = ""
    if len(changes_train[3][i]["removed_code"]) > 0:
        removed_lines = "REMOVED LINES: " + str(changes_train[3][i]["removed_code"])

    exp_prompt = f"""\n
    MODIFIED SUBSYSTEMS: {features_train.loc[i,'ns']}
    MODIFIED DIRECTORIES: {features_train.loc[i,'nd']}
    MODIFIED FILES: {features_train.loc[i,'nf']}
    ENTROPY: {features_train.loc[i,'entropy']}
    LINES OF CODE ADDED: {features_train.loc[i,'la']}
    LINES OF CODE DELETED: {features_train.loc[i,'ld']}
    LINES OF CODE IN A FILE BEFORE THE CHANGE: {features_train.loc[i,'lt']}
    FIX: {features_train.loc[i,'fix']}
    DEVELOPERS THAT CHANGED THE MOFIFIED FILES: {features_train.loc[i,'ndev']}
    AVERAGE TIME BETWEEN THE LAST AND THE CURRENT CHANGE: {features_train.loc[i,'age']}
    UNIQUE CHANGES TO THE MODIFIED FILES: {features_train.loc[i,'nuc']}
    DEVELOPER EXPERIENCE: {get_exp_desc(exp_mean, features_train.loc[i,'exp'])}
    RECENT DEVELOPER EXPERIENCE: {get_exp_desc(rexp_mean, features_train.loc[i,'rexp'])}
    DEVELOPER EXPERIANCE ON THE CURRENT SUBSYSTEM: {get_exp_desc(sexp_mean, features_train.loc[i,'sexp'])}
    """

    text = f"""COMMIT MESSAGE: {changes_train[2][i]}
              {added_lines}
               {removed_lines}
               {exp_prompt}
               """

    train_texts.append(text)

In [ ]:
len(changes_eval[0])

5465

In [ ]:
eval_texts = []

for i in range(len(changes_eval[0])):
    added_lines = ""
    if len(changes_eval[3][i]["added_code"]) > 0:
        added_lines = "ADDED LINES: " + str(changes_eval[3][i]["added_code"])

    removed_lines = ""
    if len(changes_eval[3][i]["removed_code"]) > 0:
        removed_lines = "REMOVED LINES: " + str(changes_eval[3][i]["removed_code"])

    exp_prompt = f"""\n
    MODIFIED SUBSYSTEMS: {features_eval.loc[i,'ns']}
    MODIFIED DIRECTORIES: {features_eval.loc[i,'nd']}
    MODIFIED FILES: {features_eval.loc[i,'nf']}
    ENTROPY: {features_eval.loc[i,'entropy']}
    LINES OF CODE ADDED: {features_eval.loc[i,'la']}
    LINES OF CODE DELETED: {features_eval.loc[i,'ld']}
    LINES OF CODE IN A FILE BEFORE THE CHANGE: {features_eval.loc[i,'lt']}
    FIX: {features_eval.loc[i,'fix']}
    DEVELOPERS THAT CHANGED THE MOFIFIED FILES: {features_eval.loc[i,'ndev']}
    AVERAGE TIME BETWEEN THE LAST AND THE CURRENT CHANGE: {features_eval.loc[i,'age']}
    UNIQUE CHANGES TO THE MODIFIED FILES: {features_eval.loc[i,'nuc']}
    DEVELOPER EXPERIENCE: {get_exp_desc(exp_mean, features_eval.loc[i,'exp'])}
    RECENT DEVELOPER EXPERIENCE: {get_exp_desc(rexp_mean, features_eval.loc[i,'rexp'])}
    DEVELOPER EXPERIANCE ON THE CURRENT SUBSYSTEM: {get_exp_desc(sexp_mean, features_eval.loc[i,'sexp'])}
    """

    text = f"""COMMIT MESSAGE: {changes_eval[2][i]}
              {added_lines}
               {removed_lines}
               {exp_prompt}
               """

    eval_texts.append(text)

In [ ]:
print(train_texts[0])

COMMIT MESSAGE: publish task now uses srcivypattern for ivy files finding for delivery
              ADDED LINES: ['public String getSrcivypattern ( ) {', 'File ivyFile = new File ( cache , IvyPatternHelper . substitute ( srcivypattern , organisation , module , pubRevision , "ivy" , "ivy" , "xml" ) ) ;', 'srcivypattern = artifactspattern ;', 'if ( srcivypattern = = null ) {', 'public void setSrcivypattern ( String destivypattern ) {', 'srcivypattern = destivypattern ;', 'deliver . setDeliverpattern ( getSrcivypattern ( ) ) ;', 'private String srcivypattern ;', 'return srcivypattern ;', 'Collection missing = ivy . publish ( mrid , pubRevision , cache , artifactspattern , publishResolverName , publishivy ? srcivypattern : null , doValidate ( ivy ) ) ;', '}']
               REMOVED LINES: ['deliver . setDeliverpattern ( getDeliverivypattern ( ) ) ;', 'deliverivypattern = artifactspattern ;', 'File ivyFile = new File ( cache , IvyPatternHelper . substitute ( deliverivypattern , organisatio

In [ ]:
len(train_texts), len(eval_texts), len(test_texts)

(16374, 5465, 5480)

### Labels

In [ ]:
y_true_train = []

for i in range(len(changes_train[0])):
    y_true_train.append(changes_train[1][i])

In [ ]:
y_true_eval = []

for i in range(len(changes_eval[0])):
    y_true_eval.append(changes_eval[1][i])

In [ ]:
sum(y_true_train)

1390.0

In [ ]:
sum(y_true_train)/len(y_true_train)

0.08489068034689141

In [ ]:
import pandas as pd

train_df = pd.DataFrame({'text': train_texts, 'label': y_true_train})
eval_df = pd.DataFrame({'text': eval_texts, 'label': y_true_eval})

In [ ]:
train_df.label.value_counts(normalize=True)

,proportion
label,
0.0,0.915109
1.0,0.084891


In [ ]:
# Shuffle the dataframes to ensure randomness
train_df = train_df.sample(frac=1).reset_index(drop=True)

In [ ]:
train_df.label.value_counts()

,count
label,
0.0,14984
1.0,1390


In [ ]:
!pip install datasets

In [ ]:
from datasets import DatasetDict, Dataset

# Converting pandas DataFrames into Hugging Face Dataset objects:
dataset_train = Dataset.from_pandas(train_df)
dataset_val = Dataset.from_pandas(eval_df)
dataset_test = Dataset.from_pandas(test_df)

# Combine them into a single DatasetDict
dataset = DatasetDict({
    'train': dataset_train,
    'val': dataset_val,
    'test': dataset_test
})
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16374
    })
    val: Dataset({
        features: ['text', 'label'],
        num_rows: 5465
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 5480
    })
})

In [ ]:
class_weights=(1/train_df.label.value_counts(normalize=True).sort_index()).tolist()
class_weights=torch.tensor(class_weights)
class_weights=class_weights/class_weights.sum()
class_weights

tensor([0.0849, 0.9151])

In [ ]:
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False
model.config.pretraining_tp = 1

## Testing original checkpoint

In [ ]:
import numpy as np

generate_predictions(model,test_df)
get_metrics_result(test_df)

## Model training

In [ ]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

lora_config = LoraConfig(
    r = 16,
    lora_alpha = 8,
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj'],
    lora_dropout = 0.05,
    bias = 'none',
    task_type = 'SEQ_CLS'
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

In [ ]:
def data_preprocesing(row):
    return tokenizer(row['text'], truncation=True, max_length=1024)

tokenized_data = dataset.map(data_preprocesing, batched=True,
remove_columns=['text'])
tokenized_data.set_format("torch")

Map:   0%|          | 0/16374 [00:00<?, ? examples/s]

Map:   0%|          | 0/5465 [00:00<?, ? examples/s]

Map:   0%|          | 0/5480 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding

collate_fn = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
def compute_metrics(evaluations):
    predictions, labels = evaluations
    predictions = np.argmax(predictions, axis=1)
    print({'balanced_accuracy' : balanced_accuracy_score(predictions, labels),
    'accuracy':accuracy_score(predictions,labels)})

    f1 = f1_score(labels, predictions, average="binary")
    precision = precision_score(labels, predictions, average="binary")
    recall = recall_score(labels, predictions, average="binary")
    roc_auc = roc_auc_score(labels, predictions)
    print("F1 Score:", f1)
    print("Precision:", precision)
    print("Recall:", recall)
    print("ROC AUC Score:", roc_auc)

    return {'balanced_accuracy' : balanced_accuracy_score(predictions, labels),
    'accuracy':accuracy_score(predictions,labels), "f1": f1, "precision": precision,
            "recall": recall, "roc_auc": roc_auc}

In [ ]:
from transformers import Trainer, TrainingArguments
import torch.nn.functional as F
import numpy as np

In [ ]:
class CustomTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        if class_weights is not None:
            self.class_weights = torch.tensor(class_weights,
            dtype=torch.float32).to(self.args.device)
        else:
            self.class_weights = None

    def compute_loss(self, model, inputs, return_outputs=False,
                     num_items_in_batch=None):
        labels = inputs.pop("labels").long()

        outputs = model(**inputs)

        logits = outputs.get('logits')

        if self.class_weights is not None:
            loss = F.cross_entropy(logits, labels, weight=self.class_weights)
        else:
            loss = F.cross_entropy(logits, labels)

        return (loss, outputs) if return_outputs else loss

In [ ]:
!rm -r /content/drive/MyDrive/UFPE/Tese/code_llama/auto/trained_weigths

rm: cannot remove '/content/drive/MyDrive/UFPE/Tese/code_llama/auto/trained_weigths': No such file or directory


In [ ]:
output_dir = "/content/drive/MyDrive/UFPE/Tese/code_llama/auto/trained_weigths"

In [ ]:
training_args = TrainingArguments(
    output_dir = output_dir,
    learning_rate = 1e-4,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    #per_device_train_batch_size = 4,
    #per_device_eval_batch_size = 4,
    num_train_epochs = 1,
    logging_steps=1,
    weight_decay = 0.01,
    eval_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    push_to_hub=False,
)

In [ ]:
torch.set_warn_always(False)

In [ ]:
trainer = CustomTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_data['train'],
    eval_dataset = tokenized_data['val'],
    tokenizer = tokenizer,
    data_collator = collate_fn,
    compute_metrics = compute_metrics,
    class_weights=class_weights,
)

train_result = trainer.train()

In [ ]:
trainer.save_model()
tokenizer.save_pretrained(output_dir)

In [ ]:
generate_predictions(model,test_df)
get_metrics_result(test_df)